In [4]:
# Import packages
import scanpy as sc
import pandas as pd
import random
import numpy as np
import anndata as ad
import glob
import os
from urllib.request import urlretrieve
import celltypist


print("scanpy:", sc.__version__)
print("pandas:", pd.__version__)
print("numpy:", np.__version__)
print("anndata:", ad.__version__)
print("celltypist:", celltypist.__version__)



scanpy: 1.11.1
pandas: 2.3.3
numpy: 2.3.5
anndata: 0.12.6
celltypist: 1.7.1
scipy: 1.16.3
statsmodels: 0.14.5


In [ ]:
# Initialize containers for per-file pseudobulk matrices and metadata

pb_results = []
meta_results = []

# Load a single CellTypist model 

model = celltypist.models.Model.load(
"ref_pbmc_clean_celltypist_model_AIFI_L3_2024-04-19.pkl"
)
# Collect 10x .h5 files from the data directory
h5_files = glob.glob("../data/*.h5")

# Process each file individually
for f in h5_files:
    # Derive sample ID from filename prefix before first underscore
    sample_id = os.path.basename(f).split("_")[0]
    print(f"Processing {sample_id}")

    # Read 10x Genomics HDF5 and add sample_id to cell metadata
    adata = sc.read_10x_h5(f)
    adata.var_names_make_unique()
    adata.obs["sample_id"] = sample_id

    # Preserve raw counts before any normalization
    adata.layers["counts"] = adata.X.copy()

    # Light gene filter: keep genes expressed in at least 20 cells
    sc.pp.filter_genes(adata, min_cells=20)

    # Normalize only for CellTypist
    adata.X = adata.X.astype("float32")
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)

    # Cell type annotation with CellTypist, majority voting for robustness
    pred = celltypist.annotate(
        adata,
        model=model,
        majority_voting=True
    )
    adata.obs["cell_type"] = pred.predicted_labels["predicted_labels"]

    # Restore raw count matrix for pseudobulk aggregation
    adata.X = adata.layers["counts"]

    # Per-file containers for pseudobulk vectors and metadata rows
    pb = []
    meta = []

    # Aggregate counts per annotated cell type
    for ct in adata.obs["cell_type"].unique():
        # Select cells of this cell type and count them
        idx = (adata.obs["cell_type"] == ct).to_numpy()
        n_cells = idx.sum()

        # Require at least 20 cells to form a pseudobulk
        if n_cells < 20:
            continue
        
        # Sum raw counts across cells for this cell type
        counts = adata.X[idx].sum(axis=0)

        # Store pseudobulk as a Series with gene names as index
        pb.append(pd.Series(
            np.asarray(counts).ravel(),
            index=adata.var_names,
            name=(sample_id, ct)
        ))

        # Store metadata for this pseudobulk
        sub = adata.obs.loc[idx]
        meta.append({
        "sample_id": sample_id,
        "cell_type": ct,
        "n_cells": n_cells
    })

    # Convert list of Series to a DataFrame
    pb = pd.DataFrame(pb)

    # Set a MultiIndex for rows: (sample_id, cell_type)
    pb.index = pd.MultiIndex.from_tuples(
        pb.index,
        names=["sample_id", "cell_type"]
    )

    # Accumulate results across files
    pb_results.append(pb)
    meta_results.append(pd.DataFrame(meta))

    # Delete the adata object to free up memory
    del adata  


In [16]:
# Concatenate per-file results into single DataFrames
pb_all = pd.concat(pb_results, axis=0)
meta_all = pd.concat(meta_results, axis=0, ignore_index=True)

In [17]:
# Save combined pseudobulk counts and metadata to CSV
pb_all.to_csv("pb_all.csv")
meta_all.to_csv("meta_all.csv", index=False)

In [5]:
# Read in csvs
meta_all = pd.read_csv("meta_all.csv")
pb_all = pd.read_csv("pb_all.csv")

In [9]:
# Inspect metadata to find cell types with the largest numbers of cells per pseudobulk
# Based on the sorted metadata, chose most abundant cell types to compare across samples
cell_counts = meta_all.groupby('cell_type')['n_cells'].sum()

cell_counts.sort_values(ascending=False)

cell_type
Core naive CD4 T cell    840143
GZMK- CD56dim NK cell    552613
IL1B+ CD14 monocyte      443051
Core CD14 monocyte       303364
Core naive B cell        279869
                          ...  
Memory CD8 Treg             678
CLP cell                    344
GZMK+ memory CD4 Treg       156
ISG+ MAIT                   124
BaEoMaP cell                 20
Name: n_cells, Length: 70, dtype: int64